# Automatic Differentiation

Differentiating the function
$y = 2\mathbf{x}^{\top}\mathbf{x}$
with respect to the column vector $\mathbf{x}$

In [1]:
import torch

x = torch.arange(4.0)
x

tensor([0., 1., 2., 3.])

Before we even calculate the gradient
of $y$ with respect to $\mathbf{x}$,
we will need a place to store it

In [2]:
x.requires_grad_(True)
x.grad

Now let us calculate $y$

In [3]:
y = 2 * torch.dot(x, x)
y

tensor(28., grad_fn=<MulBackward0>)

We can automatically calculate the gradient of `y`
with respect to each component of `x`

In [4]:
y.backward()
x.grad

tensor([ 0.,  4.,  8., 12.])

The gradient of the function $y = 2\mathbf{x}^{\top}\mathbf{x}$
with respect to $\mathbf{x}$ should be $4\mathbf{x}$

In [5]:
x.grad == 4 * x

tensor([True, True, True, True])

Now let us calculate another function of `x`

In [6]:
x.grad.zero_()
y = x.sum()
y.backward()
x.grad

tensor([1., 1., 1., 1.])

In deep learning
when we are calling backward on a vector,
our intent is
the sum of the partial derivatives
computed individually for each example

In [7]:
x.grad.zero_()
y = x * x
y.sum().backward()
x.grad

tensor([0., 2., 4., 6.])

Move some calculations
outside of the recorded computational graph

In [8]:
x.grad.zero_()
y = x * x
u = y.detach()
z = u * x

z.sum().backward()
x.grad == u

tensor([True, True, True, True])

In [9]:
x.grad.zero_()
y.sum().backward()
x.grad == 2 * x

tensor([True, True, True, True])

Even if
a function
required passing through a maze of Python control flow
we can still calculate the gradient of the resulting variable

In [12]:
def f(a):
    b = a * 2
    while b.norm() < 1000:
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c

a = torch.randn(size=(), requires_grad=True)
d = f(a)
d.backward()

a.grad == d / a

tensor(True)